In [1]:
import pandas as pd

## Load data from pickles

In [4]:
full_sls_books_df = pd.read_pickle('./sls_books.pkl')

In [5]:
full_gr_books_df = pd.read_pickle('./datasets/goodreads/gr_books.pkl')

In [143]:
full_sls_books_df.fillna('', inplace=True)

In [279]:
full_gr_reviews_df = pd.read_pickle('./datasets/goodreads/gr_reviews.pkl')

In [429]:
full_gr_authors_df = pd.read_pickle('./datasets/goodreads/gr_authors.pkl')

In [ ]:
# Todo
# - Full Gr authors
# - Full Gr reviews

## Data descriptions

In [8]:
print(f"Size of SLS Books Dataset: {full_sls_books_df.shape[0]}")
print(f"Size of GR Books Dataset: {full_gr_books_df.shape[0]}")

Size of SLS ISBN Map: 811649
Size of SLS Books Dataset: 1050667
Size of GR Books Dataset: 194123


## Data Cleaning

In [218]:
# Fixing BibNum to be of type string
full_sls_books_df.BibNum = full_sls_books_df.BibNum.astype(str)

In [12]:
# Removing unneeded columns
mini_sls_books_df = full_sls_books_df[['BibNum', 'Title', 'Author', 'ISBN', 'PublicationYear']].copy()

In [196]:
# Convert broken dates into empty string
full_sls_books_df.loc[(~full_sls_books_df.PublicationYear.str.match('^.*\\d{4}.*$')) & (full_sls_books_df.PublicationYear != ''), 'PublicationYear'] = ''
full_gr_books_df.loc[(~full_gr_books_df.publication_year.str.match('^\\d{4}$')) & (full_gr_books_df.publication_year != ''), 'publication_year'] = ''

In [206]:
mini_sls_books_df.PublicationYear = mini_sls_books_df.PublicationYear.str.replace(r'^.*?(\d{4}).*$', r'\1', regex=True)

## Minimzing data

In [15]:
# Removing duplicated BibNum
mini_sls_books_df.drop_duplicates(subset=['BibNum'], inplace=True)

In [20]:
# Regenrating the BibNum to ISBN map
mini_isbn_map_df = mini_sls_books_df[['BibNum', 'ISBN']].copy()
mini_isbn_map_df.ISBN = mini_isbn_map_df.ISBN.str.split(',')
mini_isbn_map_df = mini_isbn_map_df.explode(column='ISBN', ignore_index=True)
mini_isbn_map_df.ISBN = mini_isbn_map_df.ISBN.str.strip()
mini_isbn_map_df = mini_isbn_map_df[mini_isbn_map_df.ISBN != '']
mini_isbn_map_df.drop_duplicates(inplace=True)

In [66]:
mini_isbn_map_df.BibNum.nunique()

428624

In [63]:
# Delete all books from SLS that do not have a book in Goodreads
intersect_isbn_map_df = mini_isbn_map_df[mini_isbn_map_df.ISBN.isin(full_gr_books_df.isbn) | mini_isbn_map_df.ISBN.isin(full_gr_books_df.isbn13)]

In [88]:
# Remove dupliated ISBNs (room for improvement)
intersect_isbn_map_df = intersect_isbn_map_df[~intersect_isbn_map_df.ISBN.duplicated(keep=False)]

In [245]:
matched_gr_books_df = full_gr_books_df.merge(right=intersect_isbn_map_df, how='left', left_on='isbn', right_on='ISBN').drop(columns=['ISBN']).merge(right=intersect_isbn_map_df, how='left', left_on='isbn13', right_on='ISBN', suffixes=('', '13')).drop(columns=['ISBN'])

In [247]:
matched_gr_books_df.fillna('', inplace=True)

In [267]:
# Drop the books that are matching to different BibNumbers (based on isbn and isbn13)
matched_gr_books_df = matched_gr_books_df[~((matched_gr_books_df.BibNum != '') & (matched_gr_books_df.BibNum13 != '') & (matched_gr_books_df.BibNum != matched_gr_books_df.BibNum13))].copy()

AttributeError: 'DataFrame' object has no attribute 'BibNum13'

In [ ]:
# Merge BibNum and BibNum13
matched_gr_books_df.loc[matched_gr_books_df.BibNum == '', 'BibNum'] = matched_gr_books_df.BibNum13

In [266]:
# Remove BibNum13
matched_gr_books_df.drop(columns=['BibNum13'], inplace=True)

/var/folders/s4/1v4md5sx1mx4l5nwl3kvctxh0000gn/T/ipykernel_37526/2888686702.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_gr_books_df.drop(columns=['BibNum13'], inplace=True)


In [275]:
matched_gr_books_df.BibNum.str.len().value_counts()

BibNum
7    186941
6      4997
0      1068
5      1027
4        74
3         3
2         1
Name: count, dtype: int64

In [271]:
# Create map between book_id and BibNum
book_id_bibnum_map_df = matched_gr_books_df[matched_gr_books_df.BibNum != ''][['book_id', 'BibNum']].copy()

In [274]:
book_id_bibnum_map_df.to_pickle('./book_id_bibnum_map')

In [278]:
# Remove GR books that do not have a BibNum (this should not be needed in future runs)
matched_gr_books_df = matched_gr_books_df[matched_gr_books_df.BibNum != '']

In [285]:
# Remove reviews that do not have a book in the cleaned GR dataset
full_gr_reviews_df = full_gr_reviews_df[full_gr_reviews_df.book_id.isin(matched_gr_books_df.book_id)]

In [286]:
# Match the BibNum
matched_gr_reviews_df = full_gr_reviews_df.merge(right=book_id_bibnum_map_df, how='left', left_on='book_id', right_on='book_id')

In [292]:
# Add publication year from both systems
matched_gr_books_df = matched_gr_books_df.merge(right=mini_sls_books_df[['BibNum', 'PublicationYear']], how='left', left_on='BibNum', right_on='BibNum')

In [361]:
matched_gr_books_df[((matched_gr_books_df.publication_year != '') & (matched_gr_books_df.PublicationYear != '')) & (matched_gr_books_df.publication_year != matched_gr_books_df.PublicationYear)][['BibNum', 'publication_year', 'PublicationYear']].shape

(18189, 3)

In [355]:
# Remove future dates 
matched_gr_books_df.loc[matched_gr_books_df.publication_year > '2024', 'publication_year']=''

In [373]:
# Construct a publication year based on SLS year, if SLS year is empty, then fail-over to Goodreads publication year
matched_gr_books_df['final_publication_year'] = matched_gr_books_df.PublicationYear
matched_gr_books_df.loc[matched_gr_books_df.final_publication_year == '', 'final_publication_year'] = matched_gr_books_df[matched_gr_books_df.final_publication_year == ''].publication_year

In [383]:
# Manually update the records where no publication year is available in either dataset
manual_publication_years = {
    '1600981': '1994',
    '1938859': '1999',
    '2731068': '2011'
}


In [391]:
for bib, year in manual_publication_years.items():
    matched_gr_books_df.loc[matched_gr_books_df.BibNum == bib, 'final_publication_year'] = year

In [434]:
# Generating the book to author map
mini_author_map_df = matched_gr_books_df[['BibNum', 'authors']].copy()
mini_author_map_df = mini_author_map_df.explode(column='authors', ignore_index=True)
mini_author_map_df['author_id'] = mini_author_map_df.authors.str.get('author_id')
mini_author_map_df.drop(columns=['authors'], inplace=True)
mini_author_map_df.drop_duplicates(inplace=True)

In [442]:
# Drop authors that have no book in our minified dataset
mini_gr_authors_df = full_gr_authors_df[full_gr_authors_df.author_id.isin(mini_author_map_df.author_id)].copy()

In [466]:
# Cleaning the review dates

In [478]:
matched_gr_reviews_df['review_date'] = pd.to_datetime(matched_gr_reviews_df.date_updated)

/var/folders/s4/1v4md5sx1mx4l5nwl3kvctxh0000gn/T/ipykernel_37526/3770168377.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  matched_gr_reviews_df['review_date'] = pd.to_datetime(matched_gr_reviews_df.date_updated)
/var/folders/s4/1v4md5sx1mx4l5nwl3kvctxh0000gn/T/ipykernel_37526/3770168377.py:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  matched_gr_reviews_df['review_date'] = pd.to_datetime(matched_gr_reviews_df.date_updated)


In [479]:
matched_gr_reviews_df.to_pickle('./matched_gr_reviews_dated.pkl')

In [490]:
matched_gr_reviews_df['formatted_date'] = matched_gr_reviews_df.review_date.astype(str).str.slice(start=0, stop=10)

In [499]:
(matched_gr_reviews_df.formatted_date.str.match('[12]\\d{3}-[01]\\d{1}-[0123]\\d{1}')).all()

np.True_

## Exporting Data

### Books

In [512]:
exported_books_df = matched_gr_books_df[['BibNum', 'title', 'final_publication_year', 'description', 'num_pages']].rename({'BibNum': 'bib_number', 'final_publication_year': 'publication_year'}, axis=1)

In [527]:
exported_books_df.drop_duplicates(subset=['bib_number'], inplace=True)

In [528]:
exported_books_df.to_csv('./export/books.csv', index=False)

In [530]:
exported_books_df.publication_year.str.len().value_counts()

publication_year
4    185093
Name: count, dtype: int64

### Reviews

In [535]:
matched_gr_reviews_df.formatted_date.min()

'2006-08-29'

In [501]:
matched_gr_reviews_df[['BibNum', 'rating', 'formatted_date']].rename({'BibNum': 'bib_number', 'formatted_date': 'date'}, axis=1).to_csv('./export/reviews.csv', index=False)

### Authors

In [504]:
mini_gr_authors_df.columns

Index(['average_rating', 'author_id', 'text_reviews_count', 'name',
       'ratings_count'],
      dtype='object')

In [505]:
mini_gr_authors_df[['author_id', 'name']].to_csv('./export/authors.csv', index=False)

In [508]:
mini_author_map_df.rename({'BibNum': 'bib_number'}, axis=1).to_csv('./export/authored_book.csv', index=False)

In [411]:
search_bib = str(2716246)
mini_sls_books_df[mini_sls_books_df.BibNum == search_bib]

,BibNum,Title,Author,ISBN,PublicationYear
1101704,2716246,"Girl comics / writers, Colleen Coover ... [and...",,"0785147926, 0785147934, 9780785147923, 9780785...",2011


In [413]:
matched_gr_books_df[matched_gr_books_df.BibNum == search_bib][['book_id', 'BibNum', 'title', 'publication_year', 'PublicationYear', 'final_publication_year']]

,book_id,BibNum,title,publication_year,PublicationYear,final_publication_year
38183,7861285,2716246,Girl Comics,2010,2011,2011
176519,9605424,2716246,Girl Comics,2011,2011,2011
